In [1]:
import pandas as pd
import torch
import numpy as np
import os
import sys

DATA_PATH = '../../data/matches-processed/cs2/hetero-graph/'

In [3]:
concat_list = []

for i in range(0, 50):
    file_name = 100000 + i
    match_graphs = torch.load(DATA_PATH + f'{file_name}.pt', weights_only=False)
    concat_list.extend(match_graphs)
    del match_graphs

    if file_name % 10 == 9:
        torch.save(concat_list, DATA_PATH + '/concat/' + f'{file_name-9}-{file_name}.pt')
        print(f'Saved {file_name-9}-{file_name}.pt')
        del concat_list
        concat_list = []

Saved 100000-100009.pt
Saved 100010-100019.pt
Saved 100020-100029.pt
Saved 100030-100039.pt
Saved 100040-100049.pt


In [8]:
file_names = [
    '100000-100009.pt',
    '100010-100019.pt',
    '100020-100029.pt',
    '100030-100039.pt',
    '100040-100049.pt',
    '100050-100059.pt',
    '100060-100069.pt',
    '100070-100079.pt',
    '100080-100089.pt',
    '100090-100099.pt',
    '100100-100109.pt',
]

for file_name in file_names:

    match_graphs = torch.load(DATA_PATH + '/concat/' + file_name, weights_only=False)
    bomb_planted = []

    for graph in match_graphs:
        if graph.y['is_bomb_planted_at_A_site'] == np.float16(1) or graph.y['is_bomb_planted_at_B_site'] == np.float16(1):
            bomb_planted.append(graph)

    torch.save(bomb_planted, DATA_PATH + '/bomb_planted/' + file_name)
    print(f'Saved {file_name}')

Saved 100000-100009.pt
Saved 100010-100019.pt
Saved 100020-100029.pt
Saved 100030-100039.pt
Saved 100040-100049.pt
Saved 100050-100059.pt
Saved 100060-100069.pt
Saved 100070-100079.pt
Saved 100080-100089.pt
Saved 100090-100099.pt
Saved 100100-100109.pt


In [9]:
bomb_file_names = [
    '100000-100009.pt',
    '100010-100019.pt',
    '100020-100029.pt',
    '100030-100039.pt',
    '100040-100049.pt',
    '100050-100059.pt',
    '100060-100069.pt',
    '100070-100079.pt',
    '100080-100089.pt',
    '100090-100099.pt',
    '100100-100109.pt',
]

for file_name in bomb_file_names:

    match_graphs = torch.load(DATA_PATH + '/bomb_planted/' + file_name, weights_only=False)
    print(f'Graphs in {file_name}: {len(match_graphs)}')

Graphs in 100000-100009.pt: 18199
Graphs in 100010-100019.pt: 14658
Graphs in 100020-100029.pt: 17705
Graphs in 100030-100039.pt: 17674
Graphs in 100040-100049.pt: 19612
Graphs in 100050-100059.pt: 15886
Graphs in 100060-100069.pt: 16954
Graphs in 100070-100079.pt: 19228
Graphs in 100080-100089.pt: 19015
Graphs in 100090-100099.pt: 17290
Graphs in 100100-100109.pt: 15621
